In [160]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn. preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 데이터

In [161]:
# PATH = 'C:/study/keras/keras_data/stock/'            # camp PATH
PATH = 'C:/Users/nys/study/keras/keras_data/stock/'    # mypc PATH

samsung_dataset = pd.read_csv(PATH + '삼성전자 주가.csv', header=0, index_col=None, encoding='cp949', thousands = ',').loc[::-1]
amore_dataset = pd.read_csv(PATH + '아모레퍼시픽 주가.csv', header=0, index_col=None, encoding='cp949', thousands = ',').loc[::-1] 
# print(samsung)
# print(amore)

samsung_x = samsung_dataset[['종가', '저가', '고가', '기관','외인(수량)']].to_numpy()
amore_x = amore_dataset.loc[samsung_x.shape[0]:1,['종가', '저가', '고가', '기관', '외인(수량)']].to_numpy()
samsung_y = samsung_dataset[['시가']].to_numpy()
# print(samsung)
# print(samsung_y)
# print(samsung.dtypes) # int64
# print(samsung_y.dtypes) # int64
        
# print(amore.dtypes) # int64

print('<< origin >>')
print('samsung_x.shape: ', samsung_x.shape)
print('amore_x.shape: ', amore_x.shape)
print('samsung_y.shape: ', samsung_y.shape)

# --------------------------- 스케일링 --------------------------------- #
samsung_x = MinMaxScaler().fit_transform(samsung_x)
amore_x = MinMaxScaler().fit_transform(amore_x)

# --------------------------- for RNN---------------------------------- #
def split_x(dataset, timesteps):
    tmp = []
    for i in range(len(dataset) - timesteps + 1):
        subset = dataset[i : (i + timesteps)]
        tmp.append(subset)
    return np.array(tmp)

samsung_x = split_x(samsung_x, 5) 
amore_x = split_x(amore_x, 5)     
samsung_y = split_x(samsung_y, 5) 

print('<< split_x >>')
print('samsung_x.shape: ', samsung_x.shape)
print('amore._xshape: ', amore_x.shape)

samsung_x_predict = samsung_x[-1]
amore_x_predict = amore_x[-1]
samsung_x = samsung_x[:-1, :, :]
amore_x = amore_x[:-1, :, :]

samsung_x_train, samsung_x_test = train_test_split(samsung_x, train_size=0.7)
amore_x_train, amore_x_test = train_test_split(amore_x, train_size=0.7)
samsung_y_train, samsung_y_test = train_test_split(samsung_y, train_size=0.7)


print('<< train test split >>')
print('samsung_x_train.shape: ', samsung_x_train.shape, 'samsung_x_test.shape: ', samsung_x_test.shape)
print('amore_x_train.shape: ', amore_x_train.shape, 'amore_x_test.shape: ', amore_x_test.shape)
print('samsung_y_train.shape: ', samsung_y_train.shape, 'samsung_y_test.shape: ', samsung_y_test.shape)

# encoding='CP949': UTF-8 encoding error
# loc[::-1]: 행 역순 정렬 
# 여러 개의 열을 추출할 때는 대괄호 2개
# train_test_split을 위해 데이터들의 shape 맞추기


<< origin >>
samsung_x.shape:  (1980, 5)
amore_x.shape:  (1980, 5)
samsung_y.shape:  (1980, 1)
<< split_x >>
samsung_x.shape:  (1976, 5, 5)
amore._xshape:  (1976, 5, 5)
<< train test split >>
samsung_x_train.shape:  (1382, 5, 5) samsung_x_test.shape:  (593, 5, 5)
amore_x_train.shape:  (1382, 5, 5) amore_x_test.shape:  (593, 5, 5)
samsung_y_train.shape:  (1383, 5, 1) samsung_y_test.shape:  (593, 5, 1)


# 모델


In [162]:
# 삼성전자
samsung_input = Input(shape=(5, 5))
samsung_layer1 = LSTM(128, return_sequences=True,activation='relu')(samsung_input)
samsung_layer1 = Dropout(0.2)(samsung_layer1)
samsung_layer2 = LSTM(256, activation='relu')(samsung_layer1)
samsung_layer3 = Dense(1024, activation='relu')(samsung_layer2)
samsung_layer4 = Dropout(0.2)(samsung_layer3)
samsung_layer5 = Dense(512, activation='relu')(samsung_layer4)
samsung_layer6 = Dense(128, activation='relu')(samsung_layer5)
samsung_layer6 = Dense(32, activation='relu')(samsung_layer6)
samsung_output = Dense(1)(samsung_layer6)

# 아모레퍼시픽
amore_input = Input(shape=(5, 5))
amore_layer1 = LSTM(128, return_sequences=True,activation='relu')(amore_input)
amore_layer1 = Dropout(0.2)(amore_layer1)
amore_layer2 = LSTM(256, activation='relu')(amore_layer1)
amore_layer3 = Dense(1024, activation='relu')(amore_layer2)
amore_layer4 = Dropout(0.2)(amore_layer3)
amore_layer5 = Dense(512, activation='relu')(amore_layer4)
amore_layer6 = Dense(128, activation='relu')(amore_layer5)
amore_layer6 = Dense(32, activation='relu')(amore_layer6)
amore_output = Dense(1)(amore_layer6)

# 병합
merge_layer1 = concatenate([samsung_output, amore_output])
merge_layer2 = Dense(256, activation='relu')(merge_layer1)
merge_layer3 = Dense(512, activation='relu')(merge_layer2)
merge_layer4 = Dense(128, activation='relu')(merge_layer3)
merge_layer5 = Dense(32, activation='relu')(merge_layer4)
merge_output = Dense(1, activation='relu')(merge_layer5)

model = Model(inputs=[samsung_input, amore_input], outputs=[merge_output])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 5, 5)]       0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 5, 5)]       0           []                               
                                                                                                  
 lstm_40 (LSTM)                 (None, 5, 128)       68608       ['input_21[0][0]']               
                                                                                                  
 lstm_42 (LSTM)                 (None, 5, 128)       68608       ['input_22[0][0]']               
                                                                                           

# 컴파일, 훈련

In [163]:
model.compile(loss='mse', optimizer= 'adam')

ES = EarlyStopping(monitor='val_loss', mode='auto', patience=32, restore_best_weights=True)
MCP = ModelCheckpoint(monitor='val_loss', mode='auto', save_best_only=True,filepath = PATH + 'stock_ModelCheckPoint.hdf5') # 모델과 가중치 저장

model.fit([samsung_x_train, amore_x_train], samsung_y_train , epochs=1024, batch_size=1024, validation_split=0.2, callbacks=[ES, MCP])

model.save_weights(PATH + 'stock_weight.h5') # 가중치만 저장

Epoch 1/1024
2/2 [==============================] - 4s 595ms/step - loss: 1415326728192.0000 - val_loss: 1384343273472.0000
Epoch 2/1024
2/2 [==============================] - 1s 237ms/step - loss: 1415326728192.0000 - val_loss: 1384343011328.0000
Epoch 3/1024
2/2 [==============================] - 1s 316ms/step - loss: 1415326334976.0000 - val_loss: 1384341438464.0000
Epoch 4/1024
2/2 [==============================] - 1s 228ms/step - loss: 1415324499968.0000 - val_loss: 1384335409152.0000
Epoch 5/1024
2/2 [==============================] - 1s 232ms/step - loss: 1415318339584.0000 - val_loss: 1384315748352.0000
Epoch 6/1024
2/2 [==============================] - 1s 280ms/step - loss: 1415295926272.0000 - val_loss: 1384250998784.0000
Epoch 7/1024
2/2 [==============================] - 1s 246ms/step - loss: 1415224229888.0000 - val_loss: 1384040497152.0000
Epoch 8/1024
2/2 [==============================] - 1s 252ms/step - loss: 1414993674240.0000 - val_loss: 1383333888000.0000
Epoch 9/

# 평가, 예측

In [164]:
loss = model.evaluate([samsung_x_test, amore_x_test], samsung_y_test, batch_size=1024)
pred_samsung_result = model.predict(samsung_y)

print("loss(mse): :", loss)

1/1 [==============================] - 0s 80ms/step - loss: 812261048320.0000


ValueError: in user code:

    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\nys\anaconda3\envs\tf27\lib\site-packages\keras\engine\input_spec.py", line 199, in assert_input_compatibilit